# MMA 662 - Assignment 2





Installing Gurobi package:

In [1]:
# Uncomment if working with Colab:
# !pip install gurobipy
import gurobipy as gp
from gurobipy import *

import numpy as np
import pandas as pd

## 1: The Apportionment Problem.

Dataset:


In [2]:
# Population list:
Pop = np.array([5300000, 9200000, 5400000, 8000000, 3200000, 
                8800000, 8100000, 10100000, 3600000, 6200000, 
                11700000, 5800000, 7000000, 6300000, 4800000, 
                10700000, 7000000, 9400000, 6400000, 9100000])

# Quota (q_i) list:
Quota = np.array([21.80219, 37.84531, 22.21355, 32.90897, 13.16359, 
                  36.19986, 33.32033, 41.54757, 14.80903, 25.50445, 
                  48.12936, 23.859, 28.79535, 25.91581, 19.74538, 
                  44.01574, 28.79535, 38.66804, 26.32717, 37.43395])

# Province list:
Province = np.array(['P0', 'P1', 'P2', 'P3', 'P4', 
                      'P5', 'P6', 'P7', 'P8', 'P9', 
                      'P10', 'P11', 'P12', 'P13', 'P14', 
                      'P15', 'P16', 'P17', 'P18', 'P19'])

# Total seats:
s = 601

### 1.1: Minimize the sum of absolute gaps.

In [ ]:
Quota.tolist()

In [10]:
# Your solution for 1.1
P = len(Province)

model = gp.Model("Minimizing Sum Absolute Gaps")

# Defining Variables:
A = model.addVars(P, vtype=gp.GRB.INTEGER, name=["Allocation for Province "+str(i) for i in range(P)])
Z = model.addVars(P, vtype = GRB.CONTINUOUS, name = ["Absolute Error for Province "+str(i) for i in range(P)])

# Constrains:
## All seats should be filled
model.addConstr(gp.quicksum(A[i] for i in range(P)) == s, name='All seats are filled')

## Absolute value constraint:
model.addConstrs((Z[i] >= Quota[i] - A[i] for i in range(P)), name = "Absolute value for positive Gap")
model.addConstrs((Z[i] >= -(Quota[i] - A[i]) for i in range(P)), name = "Absolute value for negative Gap")


# Objective function:
model.setObjective(gp.quicksum(Z[i] for i in range(P)), GRB.MINIMIZE)


model.optimize()

for v in model.getVars():
    print(v.varname, "=", v.x)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 41 rows, 40 columns and 100 nonzeros
Model fingerprint: 0x7779cd77
Variable types: 20 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 6e+02]
Found heuristic solution: objective 1149.3456604
Presolve time: 0.00s
Presolved: 41 rows, 40 columns, 100 nonzeros
Variable types: 20 continuous, 20 integer (0 binary)

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   20 1149.34566    0.00000   100%     -    0s
H    0     0                       5.6071000    0.00000   100%     -    0s
    

### Question a) :

This linearization is now valid as both our objective function and the constraints are linear, allowing us to use a linear optimization model. The model still gives us the right answer to our problem as it "fakes" the absolute value by us implementing constraints of absolute value.

Indeed, if Quota[i] - A[i] is positive, the fact that E[i] is greater than the difference is quite obvious. But if Quota[i] - A[i] is negative, putting E[i] as greater than the difference automatically creates the same effect as the absolute value because Gurobi automatically implements the non-negativity constraint for each of the variables.

* Number of decision variables: 40 (20 for allocation and 20 for absolute error)
* Number of constraints: 41 (1 for sum(A[i])=s, 20 for "Absolute value for positive Gap" and 20 for "Absolute value for negative Gap")


### Question b) : 
The minimized Sum of Gaps for this problem is equal to 4.616000000000013

In [10]:
obj = model.getObjective()
print("b) The minimized Sum of Gaps for this problem is equal to:", obj.getValue())

b) The minimized Sum of Gaps for this problem is equal to: 4.616000000000013


### 1.2: Min-max.

In [11]:
# Your solution for 1.2
P = len(Province)
model = gp.Model("Min-max")

# Defining Variables:
A = model.addVars(P, vtype=gp.GRB.INTEGER, name=["Allocation for Province "+str(i) for i in range(P)])
Z = model.addVar(vtype = GRB.CONTINUOUS, name='Maximum Absolute Error')

# Constrains:
## All seats should be filled
model.addConstr(gp.quicksum(A[i] for i in range(P)) == s, name='All seats are filled')

## Absolute value constraint:
model.addConstrs((Z >= Quota[i] - A[i] for i in range(P)), name = "Absolute value for positive Gap")
model.addConstrs((Z >= -(Quota[i] - A[i]) for i in range(P)), name = "Absolute value for negative Gap")

# Objective function:
model.setObjective(Z, GRB.MINIMIZE)


model.optimize()

for v in model.getVars():
    print(v.varname, "=", v.x)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 41 rows, 21 columns and 100 nonzeros
Model fingerprint: 0xdd0d5cf5
Variable types: 1 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 6e+02]
Found heuristic solution: objective 574.6728301
Presolve time: 0.00s
Presolved: 41 rows, 21 columns, 100 nonzeros
Variable types: 1 continuous, 20 integer (0 binary)

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   20  574.67283    0.00000   100%     -    0s
H    0     0                       0.5475700    0.00000   100%     -    0s
H    0 

### Question a) : 

For the same reason as in 1.1, the linearization of this model is valid due to how we "faked" the absolute value function because of the Absolute value constraints implemented above in our code.

This model is similar to the one in 1.1 and gives the same results because minimizing the maximum value of the largest gap (for some i) affects all the other Z[i] variables we would have had if we coded this problem the same way we did in Problem 1.1 by therefore putting all their values below our newly created Z variable. It is therefore more efficient to solve the problem this way as we do not have to implement as many variables as in Problem 1.1.

* Number of decision variables: 21 (20 for the allocation part, and 1 for the maximum absolute error)
* Number of constraints: 41 (1 for sum(A[i])=s, 20 for "Absolute value for positive Gap" and 20 for "Absolute value for negative Gap")

### Question b):
The maximum absolute error is equal to 0.504449999999991 (see code above)

### 1.3: Political considerations.

In [7]:
# Your solution for 1.3

P = len(Province)

model = gp.Model("Minimizing Sum Absolute Gaps")

# Defining Variables:
A = model.addVars(P, vtype=gp.GRB.INTEGER, name=["Allocation for Province "+str(i) for i in range(P)])
Z = model.addVars(P, vtype = GRB.CONTINUOUS, name = ["Absolute Error for Province "+str(i) for i in range(P)])
D = model.addVar(vtype = GRB.BINARY, name='Dummy Variable')

# Constrains:
## All seats should be filled
model.addConstr(gp.quicksum(A[i] for i in range(P)) == s, name='All seats are filled')

## Absolute value constraint:
model.addConstrs((Z[i] >= Quota[i] - A[i] for i in range(P)), name = "Absolute value for positive Gap")
model.addConstrs((Z[i] >= -(Quota[i] - A[i]) for i in range(P)), name = "Absolute value for negative Gap")

##The number of seats allocated to the two least populated provinces (P4 and P8) must be equal:
model.addConstr(A[4] == A[8])

##The number of seats allocated to each of the provinces with more than 10,000,000 
model.addConstrs(A[i] <= Quota[i] for i in [7,10,15])
    
##If the number of seats allocated to P0 is more than its quota, 
#then the number of seats allocated to P2 must be more than its quota as well:
model.addConstr(A[0] >= Quota[0] - 100*(1-D))
model.addConstr(A[0] <= Quota[0] + 100*D)
model.addConstr(A[2] >= Quota[2]*D)


# Objective function:
model.setObjective(gp.quicksum(Z[i] for i in range(P)), GRB.MINIMIZE)


model.optimize()

for v in model.getVars():
    print(v.varname, "=", v.x)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 48 rows, 41 columns and 111 nonzeros
Model fingerprint: 0xa791b00f
Variable types: 20 continuous, 21 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 6e+02]
Found heuristic solution: objective 1149.3456604
Presolve removed 10 rows and 6 columns
Presolve time: 0.00s
Presolved: 38 rows, 35 columns, 94 nonzeros
Found heuristic solution: objective 1148.3456606
Variable types: 17 continuous, 18 integer (1 binary)
Found heuristic solution: objective 1147.3456606

Root relaxation: objective 3.140300e+00, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.

In [8]:
model.ObjVal

6.574919999999896

In [9]:
6.574919999999896 - 4.616000000000013

1.9589199999998828

### Question a) :
The mathematical formulas related to each consideration can be found in the code above.

* Number of Decision Variables: 41 (20 for allocation, 20 for absolute error, and 1 that is the dummy variable allowing us to form the last constraint)

* Number of constraints: 48 (The first 41 ones are the same as in Problem 1.1, and the 6 last ones are for our newly added constraints)

### Question b):
The minimum sum of absolute gaps is equal to 6.574919999999896. In problem 1.1, the minimum sum of absolute gaps was equal to 4.616000000000013, with a difference of 1.9589199999998828.

### 1.4: Opinions.

After a Google Search, I found out that the States in the US are all represented by two senators in the Senate because of an Article writen in the Constitution ([U.S. Constitution, Article I, section 3, clause 1]).

This choice is completely independent of the population per state. They therefore did not seem to have used any Optimization Program to make this decision. They did not choose only one senate because in case of death or illness, it would leave a state without any representative, and three senators would have made the Senate too crowded.

In my opinion, I think that this is a very wise choice to make, as I think that one voice is enough to represent a group of people. I don't see why a larger state in terms of population might need more representatives. They just need a good one, just as much as any other state. This was an interesting optimization problem to solve, but I personally don't think it would have been used in real life.

## 2: Did You Miss BalancedMilk Co. and Ms. Fraser?

### 2.1: The salesforce problem.

In [32]:
B = np.array([[1, 0.3, -0.1, 0, 0, -0.2, 0, -0.3, 0.4, 0.1],
              [0.2, 1, 0.1, 0.1, -0.1, 0, -0.1, 0.1, 0.3, -0.2],
              [0.2, 0, 1, 0.1, -0.1, 0.2, 0.2, 0, -0.3, 0],
              [-0.1, 0.2, 0, 1, 0.2, 0, 0.2, 0, 0.1, -0.1],
              [0.1, -0.1, -0.1, 0.1, 1, 0.1, 0.1, 0.3, 0, 0.1], 
              [-0.2, -0.2, -0.1, 0.2, 0.2, 1, 0.3, -0.1, 0, -0.2],
              [0.1, 0.2, -0.1, -0.1, 0.2, -0.1, 1, 0.2, 0.1, 0.3],
              [0.1, 0.1, 0.3, -0.1, 0.1, 0.2, 0.2, 1, 0.2, 0.1],
              [0.3, 0.4, -0.2, 0.2, 0.1, 0, 0.1, -0.1, 1, -0.1],
              [-0.1, -0.2, 0.1, 0.1, 0, -0.1, 0.2, 0.2, 0.3, 1]])
 
# Your solution for 2.1

S = len(B) #10 salesperson
Total_Award = 10000

model = gp.Model("Payoff Allocation")

# Defining variables:
A = model.addVars(S, vtype = GRB.CONTINUOUS, name=['Award allocation for salesperson'+str(i+1) for i in range(S)])
D = model.addVars(S, vtype = GRB.BINARY, name=['Award deduction for salesperson'+str(i+1) for i in range(S)])
I = model.addVars(S, vtype = GRB.BINARY, name=['Award increase for salesperson'+str(i+1) for i in range(S)])

#Constraints:
## Totality constraint:
model.addConstr(gp.quicksum(A[i] for i in range(S)) == 10000)
## Each salesperson should receive at least 1500 units of payoff (includes the two lasts constraints too):
model.addConstrs(gp.quicksum(B[i][j]*A[j] for j in range(S)) >= 1500 - 500 * D[i] + 1000 * I[i] for i in range(S))
## This limit can be dropped to 1000 units of payoff for at most three salespersons:
model.addConstr(gp.quicksum(D[i] for i in range(S)) <= 3) 
## At least three salespeople should receive 2500 or more units of payoff:
model.addConstr(gp.quicksum(I[i] for i in range(S)) >= 3)

#Objective function:
model.setObjective(gp.quicksum(gp.quicksum(B[i][j] * A[j] for j in range(S)) for i in range(S)), GRB.MAXIMIZE)

model.optimize()

for v in model.getVars():
    print(v.varname, "=", v.x)
    
payoff_B = np.empty(10)
for i in range(10): payoff_B[i] = sum(B[i][j] * A[j].X for j in range(10))
for i in range(10): print("The payoff of salesperson", (i+1), "from this allocation is %d" % payoff_B[i])
average_payoff = sum(payoff_B[i] for i in range(10)) / 10
print("The maximed average payoff is equal to", average_payoff)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 30 columns and 136 nonzeros
Model fingerprint: 0xf9d9afd8
Variable types: 10 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [9e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+04]
Presolve time: 0.00s
Presolved: 13 rows, 30 columns, 130 nonzeros
Variable types: 10 continuous, 20 integer (20 binary)

Root relaxation: objective 1.983931e+04, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 19839.3085    0    4          - 19839.3085      -     -    0s
H    0     0                    19209.432186 19839.3085  3.28%     -    0s
     0     0 19673.3457    0    4 19209.4322 19673.

### 2.2: BalancedMilk in the middle of the crisis.

In [38]:
Cost = np.array([[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
                [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
                [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
                [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
                [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
                [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
                [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
                [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]])
SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

# Your solution for 2.2
S = len(SupplyCenter)
D = len(DemandMarket)

model = gp.Model('BalancedMilk in the middle of the crisis')

# Variables:
X = model.addVars(S,D, vtype = GRB.CONTINUOUS, name = ["From "+i+" to "+j for i in SupplyCenter for j in DemandMarket])
V = model.addVars(D, name = 'Violation of Demand')
I_50 = model.addVars([2,3,4], name='50% Indicator', vtype = GRB.BINARY)
I_D = model.addVar(name='Demand Indicator', vtype = GRB.BINARY)
I_C = model.addVar(name='Competition Indicator', vtype = GRB.BINARY)

#Constraints:
## Demand and Supply logic constraints
model.addConstrs(gp.quicksum(X[i,j] for j in range(D)) <= Supply[i] for i in range(S))
model.addConstrs(gp.quicksum(X[i,j] for i in range(S)) >= Demand[j] - V[j] for j in range(D))

## 50% Capacity Constraints
model.addConstrs(gp.quicksum(X[i,j] for j in range(D)) >= 0.5*Supply[i] - 1000*(1-I_50[i]) for i in [2,3,4])
model.addConstrs(gp.quicksum(X[i,j] for j in range(D)) <= 0.5*Supply[i] + 1000*(I_50[i]) for i in [2,3,4])
model.addConstr(gp.quicksum(I_50[i] for i in [2,3,4]) >= 2)

## either D5 receives ≥ 80% of its demand, or D6 receives ≥ 60% of its demand, or both
model.addConstr(gp.quicksum(X[i,4] for i in range(S)) >= 0.8 * Demand[4] - 1000*(1-I_D))
model.addConstr(gp.quicksum(X[i,5] for i in range(S)) >= 0.6 * Demand[5] - 1000*(I_D))

## Competition constraints:
model.addConstr(gp.quicksum(X[i,1] for i in range(S)) >= 0.5 * Demand[1] - 1000*(1-I_C))
model.addConstr(gp.quicksum(X[i,1] for i in range(S)) <= 0.5 * Demand[1] + 1000*(I_C))
model.addConstr(gp.quicksum(X[i,7] for i in range(S)) >= 0.5 * Demand[7] - 1000*(1-I_C))
model.addConstr(gp.quicksum(X[i,8] for i in range(S)) >= 0.5 * Demand[8] - 1000*(1-I_C))

cost = gp.quicksum(Cost[i][j] * X[(i,j)] for i in range(S) for j in range(D))
violation = gp.quicksum(15 * V[j] for j in range(D))

model.setObjective((cost + violation), GRB.MINIMIZE)
model.optimize()

print()
print("The Total cost minimized is equal to", model.ObjVal) 
print()

print()
for v in model.getVars():
    if v.x>0:
        print(v.varname, "=", v.x)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 30 rows, 85 columns and 267 nonzeros
Model fingerprint: 0x7fd3d4d0
Variable types: 80 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+03]
Presolve removed 3 rows and 15 columns
Presolve time: 0.01s
Presolved: 27 rows, 70 columns, 207 nonzeros
Variable types: 65 continuous, 5 integer (5 binary)
Found heuristic solution: objective 17010.000000

Root relaxation: objective 1.561077e+04, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 15610.7692    0    1 17010.0000 15610.7692  8.23%     -    0s
H    0     0                    15918.0000

### 2.3 - BalancedMilk before the next crisis.

In [45]:
Cost = np.array([[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
                [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
                [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
                [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
                [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
                [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
                [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
                [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]])

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 440]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

# Your solution for 2.3

S = len(SupplyCenter)
D = len(DemandMarket)

model = gp.Model('Contingency Plan Best')

# Variables:
X = model.addVars(S,D, vtype = GRB.CONTINUOUS, name = ["Supply From "+i+" to Demand "+j for i in SupplyCenter for j in DemandMarket])
V = model.addVars(D, vtype = GRB.CONTINUOUS, name = 'Demand Violation')
ShutDown = model.addVars(S, vtype = GRB.BINARY, name=["Shut down supply center "+str(i+1) for i in range(S)])

# Supply and Demand Constraints:
model.addConstrs(gp.quicksum(X[i,j] for j in range(D)) == Supply[i] * (1 - ShutDown[i]) for i in range(S))
model.addConstrs(gp.quicksum(X[i,j] for i in range(S)) >= Demand[j] - V[j] for j in range(D))

#  BEST CASE SCENARIO:

## Adding the one-supply-center-shutdown constraint:
model.addConstr(gp.quicksum(ShutDown[i] for i in range(S)) >= 1)

cost = gp.quicksum(Cost[i][j] * X[(i,j)] for i in range(S) for j in range(D))
violation = gp.quicksum(30 * V[j] for j in range(D))

# Objective Function:
model.setObjective((cost + violation), GRB.MINIMIZE)
model.optimize()

print()
print("IN BEST CASE SCENARIO:")
print()
print("Total Cost:", model.ObjVal)
print()
for v in model.getVars():
    if v.x>0:
        print(v.varname, "=", v.x)
        
        


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 19 rows, 98 columns and 186 nonzeros
Model fingerprint: 0xbf17468d
Variable types: 90 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 0 rows and 18 columns
Presolve time: 0.00s
Presolved: 19 rows, 80 columns, 150 nonzeros
Variable types: 72 continuous, 8 integer (8 binary)
Found heuristic solution: objective 19990.000000

Root relaxation: objective 1.922583e+04, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 19225.8333    0    2 19990.0000 19225.8333  3.82%     -    0s
*    0     0               0    19460.0000